In [1]:
import numpy as np
import pandas as pd
from models.simulated_annealing import SimulatedAnnealing
import random

np.random.seed(42)

In [2]:
# 读取数据
file_path1 = '完整附件2_经济效益..xlsx'
df = pd.read_excel(file_path1, sheet_name='Sheet1')

file_path2 = '地块面积.xlsx'
area = pd.read_excel(file_path2,sheet_name='Sheet1')
bean = pd.read_excel(file_path2,sheet_name='Sheet2')

file_path3 = '可种植作物.xlsx'
judge = pd.read_excel(file_path3,sheet_name='Sheet1')

file_path4 = '2023年种植数据.xlsx'
plant_23 = pd.read_excel(file_path4,sheet_name='Sheet1')

In [3]:
# 创建一个包含年份的列表
years = list(range(2024, 2031))

# 创建一个相同长度的全零列表
zeros = [0] * len(years)

# 创建DataFrame
profit = pd.DataFrame({
    '年份': years,  # 指定列名为 'Year'
    '利润': zeros  # 指定列名为 'Value'
})

In [6]:
with pd.ExcelWriter('./2_SA/种植方案_2.xlsx', engine='openpyxl') as writer:
    for y in range(2024,2031):# 年份遍历
        # 导入答案空集
        file_path5 = 'res.xlsx'
        result1 = pd.read_excel(file_path5,sheet_name='1')
        result2 = pd.read_excel(file_path5,sheet_name='2')
        res_judge = pd.read_excel(f'./2_SA/{y-1}年种植方案.xlsx',sheet_name='Sheet1')
        for s in range(1,3): # 季度遍历
            area_temp = area[['地块类型','编号','地块面积/亩',y]]
            if(s==2):
                area_temp = area_temp[(area_temp['地块类型']== '水浇地') |(area_temp['地块类型']== '普通大棚') | (area_temp['地块类型']== '智慧大棚')]
            area_temp = area_temp.sort_values(by='地块面积/亩',ascending=False)
            for index1, row1 in area_temp.iterrows():# 地块遍历
                # 提取相应地块的农作物
                df_temp = df[df['地块类型'] == area.loc[index1,'地块类型']]
                if(y >= 2026 and bean.loc[index1,y-1] + bean.loc[index1,y-2] == 0 ):
                    df_temp = df_temp[(df_temp['作物类型'] == '蔬菜（豆类）') |( df_temp['作物类型'] == '粮食（豆类）')]
                # 根据经济效益对农作物进行排序
                df_temp = df_temp.sort_values(by='作物经济效益/(万元)',ascending=False)

                for index, row in df_temp.iterrows():# 农作物遍历
                    plant_data = plant_23[plant_23['作物名称'] == df_temp.loc[index,'作物名称']]
                    # 预期销售量浮动
                    if(df_temp.loc[index,'作物名称'] == '小麦' or df_temp.loc[index,'作物名称'] == '玉米'):
                        rate_s = random.uniform(0.05, 0.1)
                        sales = (sum(plant_data['种植面积/亩']))*((1+rate_s)**(y-2023))
                    else:
                        rate_s = random.uniform(-0.05, 0.05)
                        sales = (sum(plant_data['种植面积/亩']))*(1+rate_s)

                    # 销售价格浮动
                    rate_p = 0
                    if(df_temp.loc[index,'作物类型'] == '蔬菜' or df_temp.loc[index,'作物类型'] == '蔬菜（豆类）'):
                        rate_p = 0.05
                        price = df_temp.loc[index,'平均单价(元/斤)']*((1+0.05)**(y-2023))
                    elif(df_temp.loc[index,'作物类型'] == '食用菌'):
                        if(df_temp.loc[index,'作物名称'] == '羊肚菌'):
                            rate_p = 0.05
                            price = df_temp.loc[index,'平均单价(元/斤)']*((1-0.05)**(y-2023))
                        else:
                            rate_p = random.uniform(-0.05, -0.01)
                            price = df_temp.loc[index,'平均单价(元/斤)']*((1+rate_p)**(y-2023))
                    else:
                        price = df_temp.loc[index,'平均单价(元/斤)']    

                    # 种植成本浮动
                    cost = df_temp.loc[index,'种植成本/(元/亩)']*((1+0.05)**(y-2023))
                            
                    # 亩产量浮动
                    rate_o = random.uniform(-0.1, 0.1)
                    output = df_temp.loc[index,'亩产量/斤']*((1+rate_o)**(y-2023))

                    R = 0.5*rate_s + 0.1*rate_p + 0.4*rate_o

                    if(df_temp.loc[index,f's{s}'] ==1 and judge.loc[df_temp.loc[index,'作物编号']-1,y]<=2 and area_temp.loc[index1,y]==1 and R >= 0):
                        def obj_functon(x):

                            income = sales*output*price
                            expend = x*cost
                            discount = (x-sales)*output*price*0.5
                            return -(income - expend + discount)

                        # 最初解、上下界
                        init_sol = [0.1]
                        ub = [area_temp.loc[index1,'地块面积/亩']-area_temp.loc[index1,y]]
                        lb = [0]


                        # 退火算法开始
                        model = SimulatedAnnealing()
                        model.fit(obj_functon, init_sol, ub=ub, lb=lb, n_iters=350, init_temperature=100)
                        best_sol = model.best_sol
                        best_fitness = -model.best_costs[-1] 
                        print(df_temp.loc[index,'作物名称'],"种植量: x =", best_sol, ", 利润 =", best_fitness)  # 打印最优解

                        # 判断是否存入答案集
                        if(s==1 and res_judge.loc[index1,df_temp.loc[index,'作物名称']].any()==0):
                            df_temp.loc[index,'s2']=0
                            result1.loc[index1,df_temp.loc[index,'作物名称']] = best_sol
                            area_temp.loc[index1,y]=0
                            judge.loc[df_temp.loc[index,'作物编号']-1,y]+=1
                            if(df_temp.loc[index,'作物类型'] == '粮食（豆类）' or df_temp.loc[index,'作物类型'] == '蔬菜（豆类）'):
                                bean.loc[index1,y] = 1
                            profit.loc[y-2024,'利润'] += best_fitness
                        elif(s==2 and result1.loc[index1,df_temp.loc[index,'作物名称']]==0 and res_judge.loc[index1+28,df_temp.loc[index,'作物名称']].any()==0):
                            result2.loc[index1-26,df_temp.loc[index,'作物名称']] = best_sol
                            area_temp.loc[index1,y]=0
                            judge.loc[df_temp.loc[index,'作物编号']-1,y]+=1
                            if(df_temp.loc[index,'作物类型'] == '粮食（豆类）' or df_temp.loc[index,'作物类型'] == '蔬菜（豆类）'):
                                bean.loc[index1,y] = 1
                            profit.loc[y-2024,'利润'] += best_fitness
                        else:
                            print('该农作物',y,'年第',s,'季','不能种在',area.loc[index1,'地块名称'])
                        
        # 导出数据
        result = pd.concat([result1,result2])
        result.to_excel(writer,sheet_name=f'{y}', index=False)
        output_file_path = f'./2_SA/{y}年种植方案.xlsx'
        result.to_excel(output_file_path, sheet_name = 'Sheet1', index=False, engine='openpyxl')
        profit_path = './2_SA/年利润_2.xlsx'
        profit.to_excel(profit_path, sheet_name = 'Sheet1', index=False, engine='openpyxl')

Exception ignored in: <function ZipFile.__del__ at 0x0000025E96B05A80>
Traceback (most recent call last):
  File "d:\Python\Python311\Lib\zipfile.py", line 1870, in __del__
    self.close()
  File "d:\Python\Python311\Lib\zipfile.py", line 1887, in close
    self.fp.seek(self.start_dir)
ValueError: seek of closed file


C:\Users\田齐\AppData\Local\Temp\ipykernel_29884\277884361.py:78: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '85.91338979171435' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  result1.loc[index1,df_temp.loc[index,'作物名称']] = best_sol
C:\Users\田齐\AppData\Local\Temp\ipykernel_29884\277884361.py:83: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '190318.79173745867' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  profit.loc[y-2024,'利润'] += best_fitness


红薯 种植量: x = 85.91338979171435 , 利润 = 190318.79173745867
南瓜 种植量: x = 79.98901350423438 , 利润 = 140155.87296422222
荞麦 种植量: x = 71.99340810254063 , 利润 = 161745.15374193643


C:\Users\田齐\AppData\Local\Temp\ipykernel_29884\277884361.py:78: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '79.98901350423438' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  result1.loc[index1,df_temp.loc[index,'作物名称']] = best_sol
C:\Users\田齐\AppData\Local\Temp\ipykernel_29884\277884361.py:78: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '71.99340810254063' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  result1.loc[index1,df_temp.loc[index,'作物名称']] = best_sol


红薯 种植量: x = 67.99792477302205 , 利润 = 168179.59817688962
荞麦 种植量: x = 59.869077592126345 , 利润 = 138010.73516728586
红薯 种植量: x = 59.98535133897917 , 利润 = 151372.6720285748
红豆 种植量: x = 54.99832150759136 , 利润 = 180159.53641747174


...

C:\Users\田齐\AppData\Local\Temp\ipykernel_29884\277884361.py:85: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.5955870908674754' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  result2.loc[index1-26,df_temp.loc[index,'作物名称']] = best_sol
C:\Users\田齐\AppData\Local\Temp\ipykernel_29884\277884361.py:85: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.5915861753261616' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  result2.loc[index1-26,df_temp.loc[index,'作物名称']] = best_sol
C:\Users\田齐\AppData\Local\Temp\ipykernel_29884\277884361.py:85: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.5997528038452735' has dtype incompatible with int64, please explicitly cast to a 

羊肚菌 种植量: x = 0.5915861753261616 , 利润 = 148310.63192997294
刀豆 种植量: x = 0.5997528038452735 , 利润 = 214440.35596643403
白灵菇 种植量: x = 0.5999267566948958 , 利润 = 117576.56320838141
榆黄菇 种植量: x = 0.5999176012817579 , 利润 = 221148.59596532752
该农作物 2030 年第 2 季 不能种在 E7
羊肚菌 种植量: x = 0.5997802700846876 , 利润 = 219210.93936170184
榆黄菇 种植量: x = 0.5999908445868619 , 利润 = 287311.8521239355
羊肚菌 种植量: x = 0.5996978713664454 , 利润 = 144561.74330429197


C:\Users\田齐\AppData\Local\Temp\ipykernel_29884\277884361.py:85: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.5999908445868619' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  result2.loc[index1-26,df_temp.loc[index,'作物名称']] = best_sol


榆黄菇 种植量: x = 0.6 , 利润 = 497171.8087824355
黄心菜 种植量: x = 0.5997711146715495 , 利润 = 14009.39050587458


C:\Users\田齐\AppData\Local\Temp\ipykernel_29884\277884361.py:85: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.5997711146715495' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  result2.loc[index1-26,df_temp.loc[index,'作物名称']] = best_sol


In [ ]:
read_file_path = f'./2_SA/2023年种植方案.xlsx'
plan = pd.read_excel(read_file_path,sheet_name='Sheet1')
total = plan.sum()
total = total.drop(['地块名'])
data  = pd.DataFrame(columns=total.index)
data.loc[2023] = total
for j in range(2024,2031):
    read_file_path = f'./2_SA/{j}年种植方案.xlsx'
    plan = pd.read_excel(read_file_path,sheet_name='Sheet1')
    total = plan.sum()
    total = total.drop(['地块名'])
    data.loc[j] = total

data = data.reset_index()
year_total_file_path = './2_SA/农作物每年种植量.xlsx'
data.to_excel(year_total_file_path,sheet_name='Sheet1', index=False,engine='openpyxl')